# Data preparation

In [1]:
!pip install pymorphy2[fast]
!pip install -U pymorphy2-dicts-ru

Requirement already up-to-date: pymorphy2-dicts-ru in /usr/local/lib/python3.6/dist-packages (2.4.417127.4579844)


In [2]:
import re

import numpy as np        
import pymorphy2
from tqdm.notebook import tqdm

In [3]:
def get_data(path):
    with open(path, "r") as f:
        for line in f:
            label, headline, text = line.strip().split("\t")
            yield label, headline, text

def tokenize_text(text):
    text = text.lower()
    words = re.findall(r'\b\w+\b', text.lower())
    return words

def prepare_data(path):
    data = []
    for label, headline, text in get_data(path):
        item = {}
        item["label"] = label
        item["headline"] = tokenize_text(headline)
        item["text"] = [tokenize_text(sentance) for sentance in re.split(r"[.!?]", text) if len(sentance) > 10]
        data.append(item)
    return data

In [4]:
data_train = prepare_data("news_train.txt")
data_test = prepare_data("news_test.txt")
data_train[0]

{'headline': ['овечкин',
  'пожертвовал',
  'детской',
  'хоккейной',
  'школе',
  'автомобиль'],
 'label': 'sport',
 'text': [['нападающий',
   'вашингтон',
   'кэпиталз',
   'александр',
   'овечкин',
   'передал',
   'детской',
   'хоккейной',
   'школе',
   'автомобиль',
   'полученный',
   'им',
   'после',
   'окончания',
   'матча',
   'всех',
   'звезд',
   'национальной',
   'хоккейной',
   'лиги',
   'нхл'],
  ['об', 'этом', 'сообщается', 'на', 'официальном', 'сайте', 'лиги'],
  ['автомобиль',
   'honda',
   'accord',
   'был',
   'подарен',
   'хоккеисту',
   'по',
   'решению',
   'спонсоров',
   'мероприятия'],
  ['игрок',
   'нхл',
   'пожертвовал',
   'машину',
   'спортивной',
   'школе',
   'nova',
   'cool',
   'cats',
   'special',
   'hockey',
   'inc'],
  ['которая', 'расположена', 'в', 'штате', 'вирджиния'],
  ['овечкин',
   'общается',
   'с',
   '10',
   'летней',
   'девочкой',
   'анной',
   'шоб',
   'с',
   'синдромом',
   'дауна',
   'которая',
   'занимает

In [5]:
def postprocess_tokens_(data):
    morph = pymorphy2.MorphAnalyzer()
    for item in tqdm(data):
        item["headline"] = [morph.parse(word)[0].normal_form for word in item["headline"]] 
        item["text"] = [[morph.parse(word)[0].normal_form for word in sentance] for sentance in item["text"]] 

postprocess_tokens_(data_train)
postprocess_tokens_(data_test)

data_train[0]

{'headline': ['овечкин',
  'пожертвовать',
  'детский',
  'хоккейный',
  'школа',
  'автомобиль'],
 'label': 'sport',
 'text': [['нападать',
   'вашингтон',
   'кэпиталзти',
   'александр',
   'овечкин',
   'передать',
   'детский',
   'хоккейный',
   'школа',
   'автомобиль',
   'получить',
   'они',
   'после',
   'окончание',
   'матч',
   'весь',
   'звезда',
   'национальный',
   'хоккейный',
   'лига',
   'нхл'],
  ['о', 'это', 'сообщаться', 'на', 'официальный', 'сайт', 'лига'],
  ['автомобиль',
   'honda',
   'accord',
   'быть',
   'подарить',
   'хоккеист',
   'по',
   'решение',
   'спонсор',
   'мероприятие'],
  ['игрок',
   'нхл',
   'пожертвовать',
   'машина',
   'спортивный',
   'школа',
   'nova',
   'cool',
   'cats',
   'special',
   'hockey',
   'inc'],
  ['который', 'расположить', 'в', 'штат', 'вирджиния'],
  ['овечкин',
   'общаться',
   'с',
   '10',
   'летний',
   'девочка',
   'анна',
   'чтоб',
   'с',
   'синдром',
   'даун',
   'который',
   'заниматься',
  

# Embeddings

In [6]:
from gensim.models import Word2Vec

sentences = [item["headline"] for item in data_train]
sentences.extend([sentance for item in data_train for sentance in item["text"]])

w2v = Word2Vec(sentences, workers=8)

In [7]:
print(w2v.wv.most_similar(positive=["октябрь"]))
print(w2v.wv.most_similar(positive=["неделя"]))

[('май', 0.9765411615371704), ('сентябрь', 0.9742979407310486), ('ноябрь', 0.9734973907470703), ('февраль', 0.9723489880561829), ('декабрь', 0.9681745767593384), ('март', 0.9672043323516846), ('апрель', 0.963692307472229), ('август', 0.9618683457374573), ('июль', 0.9608937501907349), ('июнь', 0.9535524845123291)]
[('месяц', 0.8521589040756226), ('день', 0.7880641222000122), ('полгода', 0.7696123123168945), ('выходной', 0.7487046122550964), ('десятилетие', 0.7257879972457886), ('сутки', 0.6303057670593262), ('час', 0.6269206404685974), ('раз', 0.5945150852203369), ('сезон', 0.5917405486106873), ('сразу', 0.5889211893081665)]


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


# Classification

In [8]:
max_item_len = 140
label2idx = {}

def word2vec(word):
    if word in w2v.wv:
        return w2v.wv[word]
    return np.zeros((w2v.wv.vector_size,))

def prepare_data(data):
    X = []
    y = []

    for item in tqdm(data):
        label = item["label"]
        headline = item["headline"]
        text = item["text"]

        label_idx = label2idx.get(label, len(label2idx))
        label2idx[label] = label_idx

        word_idx = 0
        sent_idx = 0
        pos_in_sent = 0

        x = np.zeros(140*100)

        while word_idx < max_item_len:
            if word_idx < len(headline):
                x[100 * word_idx: 100 * word_idx + 100] = word2vec(headline[word_idx])
                word_idx += 1
            else:
                if pos_in_sent < len(text[sent_idx]):
                    x[100 * word_idx: 100 * word_idx + 100] = word2vec(text[sent_idx][pos_in_sent])
                    word_idx += 1
                    pos_in_sent += 1
                elif sent_idx < len(text) - 1:
                    sent_idx += 1
                    pos_in_sent = 0
                else:
                    word_idx += 1

        X.append(x)
        y.append(label_idx)
    
    return np.array(X), np.array(y)

X_train, y_train = prepare_data(data_train)
X_test, y_test = prepare_data(data_test)

In [9]:
X_train.shape, y_train.shape, y_train.min(), y_train.max()

((15000, 14000), (15000,), 0, 9)

In [10]:
X_test.shape, y_test.shape, y_test.min(), y_test.max()

((3000, 14000), (3000,), 0, 9)

In [11]:
def prepere_tfidf(data):
    X = []
    y = []

    for item in tqdm(data):
        label = item["label"]
        headline = item["headline"]
        text = item["text"]

        label_idx = label2idx.get(label, len(label2idx))
        label2idx[label] = label_idx

        word_idx = 0
        sent_idx = 0
        pos_in_sent = 0

        x = []

        while word_idx < max_item_len:
            if word_idx < len(headline):
                x.append(headline[word_idx])
                word_idx += 1
            else:
                if pos_in_sent < len(text[sent_idx]):
                    x.append(text[sent_idx][pos_in_sent])
                    word_idx += 1
                    pos_in_sent += 1
                elif sent_idx < len(text) - 1:
                    sent_idx += 1
                    pos_in_sent = 0
                else:
                    x.append("PLACEHOLDER")
                    word_idx += 1

        X.append(" ".join(x))
        y.append(label_idx)
    
    return X, y

X_train_idf, y_train = prepere_tfidf(data_train)
X_test_idf, y_test = prepere_tfidf(data_test)

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
X_train_idf = tfidf.fit_transform(X_train_idf)
X_test_idf = tfidf.transform(X_test_idf)

In [13]:
X_train_idf.shape

(15000, 75061)

In [14]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(n_jobs=-1)
clf.fit(X_train_idf, y_train)
preds = clf.predict(X_test_idf)
print(f"accuracy = {(y_test == preds).mean()}")

accuracy = 0.859


In [15]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB()
clf.fit(X_train_idf, y_train)
preds = clf.predict(X_test_idf)
print(f"accuracy = {(y_test == preds).mean()}")

accuracy = 0.798
